In [1]:
//%maven org.nd4j:nd4j-cuda-9.0:1.0.0-beta
//%maven org.deeplearning4j:deeplearning4j-cuda-9.0:1.0.0-beta
//%maven org.datavec:datavec-api:1.0.0-beta

%maven org.nd4j:nd4j-native-platform:1.0.0-beta
%maven org.deeplearning4j:deeplearning4j-core:1.0.0-beta
%maven org.datavec:datavec-api:1.0.0-beta

In [7]:
import org.deeplearning4j.datasets.iterator.*;
import org.deeplearning4j.datasets.iterator.impl.*;
import org.deeplearning4j.nn.api.*;
import org.deeplearning4j.nn.multilayer.*;
import org.deeplearning4j.nn.graph.*;
import org.deeplearning4j.nn.conf.*;
import org.deeplearning4j.nn.conf.inputs.*;
import org.deeplearning4j.nn.conf.layers.*;
import org.deeplearning4j.nn.weights.*;
import org.deeplearning4j.optimize.listeners.*;
import org.deeplearning4j.datasets.datavec.RecordReaderMultiDataSetIterator;
import org.deeplearning4j.eval.Evaluation;

import org.nd4j.linalg.learning.config.*; // for different updaters like Adam, Nesterovs, etc.
import org.nd4j.linalg.activations.Activation; // defines different activation functions like RELU, SOFTMAX, etc.
import org.nd4j.linalg.lossfunctions.LossFunctions;
import org.nd4j.linalg.indexing.conditions.GreaterThan

In [3]:
import org.nd4j.linalg.dataset.DataSet;
import org.nd4j.linalg.dataset.MiniBatchFileDataSetIterator;
import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.factory.Nd4j;
import org.nd4j.linalg.dataset.api.iterator.DataSetIterator;

In [34]:
INDArray features = Nd4j.rand(new int[]{100, 1, 20, 20}).mul(5);
INDArray ones = Nd4j.ones(new int[]{100, 1, 20, 20}).mul(22);
INDArray labels = Nd4j.rand(100, 1);

In [35]:
features.mul(features.lte(3)).add(features.gt(3).mul(3))

[[[[    2.1677,    0.3654,    0.3643  ...    1.3710    0.8520,    3.0000], 
   [    3.0000,    3.0000,    1.5545  ...    0.0757    3.0000,    2.0436], 
   [    3.0000,    3.0000,    3.0000  ...    1.0942    1.7830,    2.0918], 
    ..., 
   [    3.0000,    0.0809,    2.2502  ...    1.2629    2.7562,    3.0000], 
   [    1.0567,    2.5756,    1.4896  ...    3.0000    3.0000,    3.0000], 
   [    0.3929,    1.6279,    3.0000  ...    2.9354    3.0000,    3.0000]]], 


 [[[    3.0000,    3.0000,    3.0000  ...    1.8164    1.9472,    3.0000], 
   [    1.3856,    1.3345,    3.0000  ...    3.0000    3.0000,    2.9744], 
   [    1.5613,    3.0000,    0.7618  ...    2.7281    0.2013,    0.6775], 
    ..., 
   [    1.2874,    2.5631,    2.2345  ...    3.0000    3.0000,    3.0000], 
   [    1.8011,    0.5864,    3.0000  ...    0.9495    3.0000,    3.0000], 
   [    3.0000,    0.9113,    2.4567  ...    1.7606    2.3075,    2.1296]]], 


 [[[    1.1895,    1.0417,    1.9304  ...    2.0708    1.058

In [5]:
features = features.add(features.mean().neg()).div(features.std())

In [6]:
DataSet d = new DataSet(features, labels);

In [7]:
DataSetIterator di = new ListDataSetIterator(d.asList(), 10);

In [8]:
MultiLayerConfiguration conf = new NeuralNetConfiguration.Builder()
    .seed(123)
    .optimizationAlgo(OptimizationAlgorithm.STOCHASTIC_GRADIENT_DESCENT)
    .list()
    .layer(0, new ConvolutionLayer.Builder().kernelSize(3, 3).stride(1, 1).nIn(1).nOut(20).dropOut(.5).build())
    .layer(1, new OutputLayer.Builder().nOut(1).weightInit(WeightInit.XAVIER).build())
    .setInputType(InputType.convolutional(20, 20, 1))
    .pretrain(false).backprop(true)
    .build()

In [9]:
MultiLayerNetwork network = new MultiLayerNetwork(conf);
network.init();
network.setListeners(new ScoreIterationListener(1));

[IJava-executor-0] INFO org.deeplearning4j.nn.multilayer.MultiLayerNetwork - Starting MultiLayerNetwork with WorkspaceModes set to [training: ENABLED; inference: ENABLED], cacheMode set to [NONE]


In [10]:
for (int i=0; i<3; i++) {
    network.fit(di);
    System.out.println("*** Completed epoch " + i + ".");
}

[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 0 is 0.18465788364410402
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 1 is 0.2753016710281372
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 2 is 0.12080224752426147
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 3 is 0.19649918079376222
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 4 is 0.11572840213775634
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 5 is 0.06093320846557617
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 6 is 0.07324883937835694
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener

*** Completed epoch 0.


[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 10 is 0.03940052688121796
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 11 is 0.07562063932418824
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 12 is 0.035659310221672055
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 13 is 0.07124021053314208
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 14 is 0.05120176672935486
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 15 is 0.026362788677215577
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 16 is 0.03158822357654571
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterati

*** Completed epoch 1.


[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 20 is 0.023371633887290955
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 21 is 0.03858671486377716
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 22 is 0.019564956426620483
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 23 is 0.041256952285766604
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 24 is 0.028369739651679993
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 25 is 0.015617898106575013
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIterationListener - Score at iteration 26 is 0.021478107571601866
[IJava-executor-0] INFO org.deeplearning4j.optimize.listeners.ScoreIte

*** Completed epoch 2.


In [8]:
String tmp_dir = System.getProperty("java.io.tmpdir");


/tmp

In [21]:
import org.datavec.image.loader.NativeImageLoader;

In [23]:
NativeImageLoader loader = new NativeImageLoader(28, 28, 3);
INDArray image = loader.asMatrix(new File("/home/bzr0014/Pictures/LSU_HIP.png"));

In [29]:
Arrays.toString(image.shape())

[1, 3, 28, 28]

In [30]:
int seed = 2345;
        double learningRate = 0.05;
        int numInputs = 2;   // x and y.
        int numHiddenNodes = 100;
        int numOutputs = 3 ; //R, G and B value.

        MultiLayerConfiguration conf = new NeuralNetConfiguration.Builder()
            .seed(seed)
            .weightInit(WeightInit.XAVIER)
            .updater(new Nesterovs(learningRate, 0.9))
            .list()
            .layer(0, new DenseLayer.Builder().nIn(numInputs).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(1, new DenseLayer.Builder().nIn(numHiddenNodes).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(2, new DenseLayer.Builder().nIn(numHiddenNodes).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(3, new DenseLayer.Builder().nIn(numHiddenNodes).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(4, new DenseLayer.Builder().nIn(numHiddenNodes).nOut(numHiddenNodes)
                .activation(Activation.LEAKYRELU)
                .build())
            .layer(5, new OutputLayer.Builder(LossFunctions.LossFunction.L2)
                .activation(Activation.IDENTITY)
                .nIn(numHiddenNodes).nOut(numOutputs).build())
            .build();

        MultiLayerNetwork net = new MultiLayerNetwork(conf);
        net.init();

In [36]:
net.getLayerNames()

[layer0, layer1, layer2, layer3, layer4, layer5]

In [42]:
import org.deeplearning4j.nn.api.Layer

In [6]:
NativeImageLoader loader = new NativeImageLoader(200, 200, 3);


In [4]:
import org.datavec.image.loader.NativeImageLoader;
import org.nd4j.linalg.api.ndarray.INDArray;
import org.nd4j.linalg.factory.Nd4j

In [7]:
INDArray image = loader.asMatrix(new File(
        "/Users/behnamrasoolian/Dropbox/Photos/IMG_20140220_034619288.jpg")) 

[IJava-executor-0] INFO org.nd4j.linalg.factory.Nd4jBackend - Loaded [CpuBackend] backend
[IJava-executor-0] INFO org.nd4j.nativeblas.NativeOpsHolder - Number of threads used for NativeOps: 2
[IJava-executor-0] INFO org.nd4j.nativeblas.Nd4jBlas - Number of threads used for BLAS: 2
[IJava-executor-0] INFO org.nd4j.linalg.api.ops.executioner.DefaultOpExecutioner - Backend used: [CPU]; OS: [Mac OS X]
[IJava-executor-0] INFO org.nd4j.linalg.api.ops.executioner.DefaultOpExecutioner - Cores: [4]; Memory: [1.0GB];
[IJava-executor-0] INFO org.nd4j.linalg.api.ops.executioner.DefaultOpExecutioner - Blas vendor: [OPENBLAS]


In [8]:
INDArray image = loader.asMatrix(new File("/Users/behnamrasoolian/Dropbox/Photos/IMG_20140220_034619288.jpg"))

In [14]:
loader.asMat(image)

org.bytedeco.javacpp.opencv_core$Mat[width=200,height=200,depth=32,channels=3]